In [16]:
%reload_ext autoreload
%autoreload 2

In [17]:
import numpy as np 
from pathlib import Path
from rtseg.utils.param_io import load_params
import matplotlib.pyplot as plt
%matplotlib qt5
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull, Delaunay
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import h5py

In [18]:
param = load_params(Path('C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0126/expt_params.yaml'), ref_type='expt')
param.Save.directory = 'C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0126/'
save_dir = Path(param.Save.directory)

In [19]:
with h5py.File(save_dir / 'fork_score_data.h5', 'r') as hdf:
    fork_data = {}
    for key in hdf['fork_data'].keys():
        fork_data[key] = hdf['fork_data'][key][()]

flat_heatmap_init_pos_traps = fork_data['flat_heatmap_around_init_pos_traps']
print(fork_data['heatmap_around_init_pos_traps'].shape)
nr_pix_fork = flat_heatmap_init_pos_traps.shape[0]

(10, 34, 60, 28)


In [20]:
flat_heatmap_init_re = flat_heatmap_init_pos_traps.reshape(nr_pix_fork, -1)
non_zero_ind = np.logical_and(np.all(~np.isnan(flat_heatmap_init_re), axis=0), np.any(flat_heatmap_init_re > 0, axis=0))
non_zero_flat_heatmap_init = np.transpose(flat_heatmap_init_re[:, non_zero_ind])

In [21]:
covar = np.cov(non_zero_flat_heatmap_init, rowvar=False)
_, eigvecs = np.linalg.eig(covar)
selected_eigvecs = np.fliplr(eigvecs[:,0:3])
data_proj = non_zero_flat_heatmap_init @ selected_eigvecs
data_proj[:,1] = -data_proj[:,1]
print(covar.shape, eigvecs.shape, data_proj.shape)

(340, 340) (340, 340) (1675, 3)


In [22]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(data_proj[:,0], data_proj[:,1], data_proj[:,2], marker='o')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.view_init(elev=20)


In [23]:
ref_dir = Path('C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0125/')
with h5py.File(ref_dir / 'pca_data_fork_plots.h5', 'r') as hdf:
    ref_selected_eigvecs = hdf['ref_selected_eigvecs'][()]

In [13]:
print(ref_selected_eigvecs.shape, non_zero_flat_heatmap_init.shape, selected_eigvecs.shape)

(272, 3) (1675, 340) (340, 3)


In [15]:
test = non_zero_flat_heatmap_init @ ref_selected_eigvecs

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 272 is different from 340)